In [1]:
# collect_top10_hourly.py
from pathlib import Path

from data_pull import (
    BinanceData,
    top_market_cap_spot_symbols,
    collect_spot_market_data,
    fetch_full_hourly_klines,
)

OUTPUT_ROOT = Path("data") / "spot_top10_hourly"


def main():
    client = BinanceData()  # loads BINANCE_API_KEY / BINANCE_API_SECRET from .env if present
    symbols = top_market_cap_spot_symbols(quote_asset="USDT", limit=10)
    if not symbols:
        raise SystemExit("No market-cap symbols returned; check connectivity or Binance API status.")

    OUTPUT_ROOT.mkdir(parents=True, exist_ok=True)

    for symbol in symbols:
        print(f"[{symbol}] snapshot + full hourly history")
        symbol_dir = OUTPUT_ROOT / symbol
        symbol_dir.mkdir(parents=True, exist_ok=True)

        # spot snapshot (depth, trades, tickers, 1h klines, exchange info)
        collect_spot_market_data(
            client,
            symbol,
            output_root=OUTPUT_ROOT,
            depth_limit=500,
            trades_limit=1000,
            agg_trades_limit=1000,
            kline_intervals=["1h"],
            kline_limit=1000,
            open_interest_period="1h",
            open_interest_limit=0,  # skip futures extras; keep it spot-only
        )

        # full all-time hourly candle history
        history = fetch_full_hourly_klines(client, symbol, interval="1h")
        if history.empty:
            print(f"[{symbol}] no hourly klines returned")
            continue

        history_path = symbol_dir / "klines_1h_full_history.csv"
        history.to_csv(history_path, index=False)
        print(f"[{symbol}] saved {len(history)} hourly bars -> {history_path}")


if __name__ == "__main__":
    main()


[BTCUSDT] snapshot + full hourly history
[BTCUSDT] saved 71669 hourly bars -> data\spot_top10_hourly\BTCUSDT\klines_1h_full_history.csv
[ETHUSDT] snapshot + full hourly history
[ETHUSDT] saved 71669 hourly bars -> data\spot_top10_hourly\ETHUSDT\klines_1h_full_history.csv
[BNBUSDT] snapshot + full hourly history
[BNBUSDT] saved 69732 hourly bars -> data\spot_top10_hourly\BNBUSDT\klines_1h_full_history.csv
[XRPUSDT] snapshot + full hourly history
[XRPUSDT] saved 65465 hourly bars -> data\spot_top10_hourly\XRPUSDT\klines_1h_full_history.csv
[SOLUSDT] snapshot + full hourly history
[SOLUSDT] saved 45615 hourly bars -> data\spot_top10_hourly\SOLUSDT\klines_1h_full_history.csv
[DOGEUSDT] snapshot + full hourly history
[DOGEUSDT] saved 55257 hourly bars -> data\spot_top10_hourly\DOGEUSDT\klines_1h_full_history.csv
[TRXUSDT] snapshot + full hourly history
[TRXUSDT] saved 64550 hourly bars -> data\spot_top10_hourly\TRXUSDT\klines_1h_full_history.csv
[ADAUSDT] snapshot + full hourly history
[ADA